In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials

from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager

from datetime import datetime, timedelta

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

setup_logging(
    level=logging.DEBUG,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials)

In [ ]:
await dxlink.subscribe_to_candles(
    symbol="BTC/USD:CXTALP", interval="5m", from_time=datetime.now() - timedelta(minutes=10)
)

In [ ]:
# Subscribe to 5-minute candles for SPY
await dxlink.unsubscribe_to_candles(symbol="BTC/USD:CXTALP", interval="5m")

In [ ]:
symbols = ["BTC/USD:CXTALP", "SPY", "NVDA"]
await dxlink.subscribe(symbols)

In [ ]:
await dxlink.unsubscribe(symbols)

In [ ]:
# Access candle data
dxlink.router.handler[Channels.Candle].processors["feed"].df.sort_values(
    "timestamp", ascending=True
)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df

In [ ]:
await dxlink.close()

In [17]:
# Notes

# We don't need to keep all the Candlestick ticks, just the latest "Live" tick preceded by a hsitory of prior candlesticks